#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [7]:
ust = pd.read_csv('users_table.csv')
ust.tail()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
40320,55743,1,2014-09-13 21:03:50,AussieMeg,2014-09-13 21:18:52,NaN,NaN,NaN,0,0,0,5026902,NaN,http://graph.facebook.com/665821703/picture?ty...
40321,55744,6,2014-09-13 21:39:30,Mia Maria,2014-09-13 21:39:30,NaN,NaN,NaN,1,0,0,5026998,NaN,NaN
40322,55745,101,2014-09-13 23:45:27,tronbabylove,2014-09-13 23:45:27,NaN,United States,NaN,0,0,0,481766,NaN,https://www.gravatar.com/avatar/faa7a3fdbd8308...
40323,55746,106,2014-09-14 00:29:41,GPP,2014-09-14 02:05:17,NaN,NaN,"<p>Stats noobie, product, marketing &amp; medi...",1,0,0,976289,NaN,https://www.gravatar.com/avatar/6d9e9fa6b783a3...
40324,55747,1,2014-09-14 01:01:44,Shivam Agrawal,2014-09-14 01:19:04,NaN,India,<p>Maths Enthusiast </p>\r\n,0,0,0,5027354,NaN,https://lh4.googleusercontent.com/-ZsXhwVaFmiY...


#### 3. Rename Id column to userId

In [10]:
ust = ust.rename(columns = {'Id': 'userId'})
ust.head()

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,NaN
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\r\n\r\n<p>...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,NaN


#### 4. Import posts table:

In [11]:
pst = pd.read_csv('posts_table.csv')
pst.tail()

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN
39999,48325,1,NaN,2013-01-23 09:44:07,-1,116.0,"<p>I have 10 vectors each having 100,000 point...",19968.0,2013-02-22 11:23:54,are data sets obtained from a Normal distribut...,...,2.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [27]:
pst = pst.rename(columns = {'Id':'postId', 'OwnerUserId':'userId'})
pst.head(1)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [21]:
users_columns = ust[{'userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes'}]
users_columns


,Reputation,Views,DownVotes,UpVotes,userId
0,1,0,1920,5007,-1
1,101,25,0,3,2
2,101,22,0,19,3
3,101,11,0,0,4
4,6792,1145,5,662,5
...,...,...,...,...,...
40320,1,0,0,0,55743
40321,6,1,0,0,55744
40322,101,0,0,0,55745
40323,106,1,0,0,55746


In [29]:
posts_columns = pst[{'postId', 'Score','userId', 'ViewCount' , 'CommentCount'}]
posts_columns.head()

,CommentCount,Score,ViewCount,postId,userId
0,1,23,1278.0,1,8.0
1,1,22,8198.0,2,24.0
2,4,54,3613.0,3,18.0
3,2,13,5224.0,4,23.0
4,3,81,NaN,5,23.0


#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [32]:
df_pu = users_columns.merge(posts_columns)
df_pu.head()

,Reputation,Views,DownVotes,UpVotes,userId,CommentCount,Score,ViewCount,postId
0,1,0,1920,5007,-1,0,0,NaN,2175
1,1,0,1920,5007,-1,0,0,NaN,8576
2,1,0,1920,5007,-1,0,0,NaN,8578
3,1,0,1920,5007,-1,0,0,NaN,8981
4,1,0,1920,5007,-1,0,0,NaN,8982


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [41]:
null_values = df_pu.isnull().sum()
zeros_count = df_pu[df_pu == 0].count()
ones_count = df_pu[df_pu == 1].count() #Addicted on this rs!
print(null_values)
print(zeros_count)
print(ones_count)

Reputation          0
Views               0
DownVotes           0
UpVotes             0
userId              0
CommentCount        0
Score               0
ViewCount       23572
postId              0
dtype: int64
Reputation          0
Views            1150
DownVotes       18036
UpVotes          6472
userId              0
CommentCount    14983
Score            4652
ViewCount           0
postId              0
dtype: int64
Reputation       707
Views           1189
DownVotes       2242
UpVotes         1270
userId             0
CommentCount    6780
Score           8030
ViewCount          0
postId             1
dtype: int64


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [52]:
#Fill it's better because in this case analyze the number of users without comment, score or view can give some insight about how many users are acctualy "using".

df_pu['ViewCount'] = df_pu['ViewCount'].fillna(0)
df_pu.head()





,Reputation,Views,DownVotes,UpVotes,userId,CommentCount,Score,ViewCount,postId
0,1,0,1920,5007,-1,0,0,0.0,2175
1,1,0,1920,5007,-1,0,0,0.0,8576
2,1,0,1920,5007,-1,0,0,0.0,8578
3,1,0,1920,5007,-1,0,0,0.0,8981
4,1,0,1920,5007,-1,0,0,0.0,8982


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [54]:
df_pu.dtypes

Reputation        int64
Views             int64
DownVotes         int64
UpVotes           int64
userId            int64
CommentCount      int64
Score             int64
ViewCount       float64
postId            int64
dtype: object

In [55]:
#ViewCount needs to be a Integer and userId and postId shoudn't be changed so it's better change dtype for object
df_pu['userId'] = df_pu['userId'].astype('object')
df_pu['postId'] = df_pu['postId'].astype('object')
df_pu['ViewCount'] = df_pu['ViewCount'].astype('int64')
df_pu.dtypes

Reputation       int64
Views            int64
DownVotes        int64
UpVotes          int64
userId          object
CommentCount     int64
Score            int64
ViewCount        int64
postId          object
dtype: object